In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5/starfish_yolov5/Arial.ttf /root/.config/Ultralytics/

In [ ]:
shutil.copytree('/kaggle/input/yolov5/starfish_yolov5', '/kaggle/working/yolov5/starfish_yolov5')
os.chdir('/kaggle/working/yolov5/starfish_yolov5') # install dependencies

In [ ]:
file = open("/kaggle/input/starfish-fold/video/video/val_video1.txt")
lines = file.readlines()
# ../input/starfish-fold/video/video/val_video1.txt

In [ ]:
%%writefile fold0.yaml

names: [starfish]
nc: 1
train: '/kaggle/input/starfish-fold/video/video/val_video1.txt'
val: '/kaggle/input/starfish-fold/video/video/val_video1.txt'

In [ ]:
# ../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt

**copy from  yolov5/val.py**

In [ ]:
import argparse
import json
import os
import sys
from pathlib import Path
from threading import Thread
import numpy as np
import torch
from tqdm import tqdm

ROOT= '/kaggle/working/yolov5/starfish_yolov5/'
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT)) 

from models.experimental import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_img_size, check_requirements, \
    check_suffix, check_yaml, box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, \
    increment_path, colorstr, print_args
from utils.metrics import ap_per_class, ConfusionMatrix
from utils.plots import output_to_target, plot_images, plot_val_study
from utils.torch_utils import select_device, time_sync
from utils.callbacks import Callbacks

def save_one_txt(predn, save_conf, shape, file):
    # Save one txt result
    gn = torch.tensor(shape)[[1, 0, 1, 0]]  # normalization gain whwh
    for *xyxy, conf, cls in predn.tolist():
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
        with open(file, 'a') as f:
            f.write(('%g ' * len(line)).rstrip() % line + '\n')


def save_one_json(predn, jdict, path, class_map):
    # Save one JSON result {"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}
    image_id = int(path.stem) if path.stem.isnumeric() else path.stem
    box = xyxy2xywh(predn[:, :4])  # xywh
    box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
    for p, b in zip(predn.tolist(), box.tolist()):
        jdict.append({'image_id': image_id,
                      'category_id': class_map[int(p[5])],
                      'bbox': [round(x, 3) for x in b],
                      'score': round(p[4], 5)})


def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = torch.zeros(detections.shape[0], iouv.shape[0], dtype=torch.bool, device=iouv.device)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    x = torch.where((iou >= iouv[0]) & (labels[:, 0:1] == detections[:, 5]))  # IoU above threshold and classes match
    if x[0].shape[0]:
        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detection, iou]
        if x[0].shape[0] > 1:
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
            # matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
        matches = torch.Tensor(matches).to(iouv.device)
        correct[matches[:, 1].long()] = matches[:, 2:3] >= iouv
    return correct

In [ ]:
# ../input/starfish-yolo/best_fold1_f2_64017.pt
@torch.no_grad()
def run(data = '/kaggle/working/yolov5/starfish_yolov5/fold0.yaml',
        weights='/kaggle/input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',  # model.pt path(s)
        batch_size=2,  # batch size
        imgsz=9000,  # inference size (pixels)
        conf_thres=0.001,  # confidence threshold
        iou_thres=0.45,  # NMS IoU threshold
        task='val',  # train, val, test, speed or study
        device='0',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        single_cls=False,  # treat as single-class dataset
        augment=True,  # augmented inference
        verbose=False,  # verbose output
        save_txt=False,  # save results to *.txt
        save_hybrid=False,  # save label+prediction hybrid results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_json=False,  # save a COCO-JSON results file
        project=ROOT + 'runs/val',  # save to project/name
        name='exp',  # save to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        half=True,  # use FP16 half-precision inference
        model=None,
        dataloader=None,
        save_dir=Path(''),
        plots=True,
        callbacks=Callbacks(),
        compute_loss=None,
        ):
    # Initialize/load model and set device
    training = model is not None
    if training:  # called by train.py
        device = next(model.parameters()).device  # get model device

    else:  # called directly
        device = select_device(device, batch_size=batch_size)

        # Directories
        save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
        (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

        # Load model
        check_suffix(weights, '.pt')
        model = attempt_load(weights, map_location=device)  # load FP32 model
        gs = max(int(model.stride.max()), 32)  # grid size (max stride)
        imgsz = check_img_size(imgsz, s=gs)  # check image size

        # Multi-GPU disabled, incompatible with .half() https://github.com/ultralytics/yolov5/issues/99
        # if device.type != 'cpu' and torch.cuda.device_count() > 1:
        #     model = nn.DataParallel(model)

        # Data
        data = check_dataset(data)  # check

    # Half
    half &= device.type != 'cpu'  # half precision only supported on CUDA
    model.half() if half else model.float()

    # Configure
    model.eval()
    is_coco = isinstance(data.get('val'), str) and data['val'].endswith('coco/val2017.txt')  # COCO dataset
    nc = 1 if single_cls else int(data['nc'])  # number of classes
    iouv = torch.linspace(0.3, 0.8, 11).to(device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()

    # Dataloader
    if not training:
        if device.type != 'cpu':
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
        pad = 0.0 if task == 'speed' else 0.5
        task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
        dataloader = create_dataloader(data[task], imgsz, batch_size, gs, single_cls, pad=pad, rect=True,
                                       prefix=colorstr(f'{task}: '))[0]

    seen = 0
    confusion_matrix = ConfusionMatrix(nc=nc)
    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    class_map = coco80_to_coco91_class() if is_coco else list(range(1000))
    s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    dt, p, r, f1, mp, mr, map50, map = [0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class = [], [], [], []
    for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
        t1 = time_sync()
        img = img.to(device, non_blocking=True)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        targets = targets.to(device)
        nb, _, height, width = img.shape  # batch size, channels, height, width
        t2 = time_sync()
        dt[0] += t2 - t1

        # Run model
        out, train_out = model(img, augment=augment)  # inference and training outputs
        dt[1] += time_sync() - t2

        # Compute loss
        if compute_loss:
            loss += compute_loss([x.float() for x in train_out], targets)[1]  # box, obj, cls

        # Run NMS
        targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
        lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
        t3 = time_sync()
        out = non_max_suppression(out, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls)
        dt[2] += time_sync() - t3

        # Statistics per image
        for si, pred in enumerate(out):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            path, shape = Path(paths[si]), shapes[si][0]
            seen += 1

            if len(pred) == 0:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Predictions
            if single_cls:
                pred[:, 5] = 0
            predn = pred.clone()
            scale_coords(img[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

            # Evaluate
            if nl:
                tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
                scale_coords(img[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
                labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
                correct = process_batch(predn, labelsn, iouv)
                if plots:
                    confusion_matrix.process_batch(predn, labelsn)
            else:
                correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))  # (correct, conf, pcls, tcls)

            # Save/log
            if save_txt:
                save_one_txt(predn, save_conf, shape, file=save_dir / 'labels' / (path.stem + '.txt'))
            if save_json:
                save_one_json(predn, jdict, path, class_map)  # append to COCO-JSON dictionary
            callbacks.run('on_val_image_end', pred, predn, path, names, img[si])

        # Plot images
        if plots and batch_i < 3:
            f = save_dir / f'val_batch{batch_i}_labels.jpg'  # labels
            Thread(target=plot_images, args=(img, targets, paths, f, names), daemon=True).start()
            f = save_dir / f'val_batch{batch_i}_pred.jpg'  # predictions
            Thread(target=plot_images, args=(img, output_to_target(out), paths, f, names), daemon=True).start()

    # Compute statistics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        p, r, ap, f2, ap_class,conf_i = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=names)
        ap50, ap = ap[:, 4], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%11i' * 2 + '%11.3g' * 4  # print format
    print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))
    print('conf:' + str(conf_i*0.001))
    f2 = f2[0]
    print('f2-score:'+str(f2))

    # Print results per class
    if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            print(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

    # Print speeds
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    if not training:
        shape = (batch_size, 3, imgsz, imgsz)
        print(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {shape}' % t)

    # Plots
    if plots:
        confusion_matrix.plot(save_dir=save_dir, names=list(names.values()))
        callbacks.run('on_val_end')

    # Save JSON
    if save_json and len(jdict):
        w = Path(weights[0] if isinstance(weights, list) else weights).stem if weights is not None else ''  # weights
        anno_json = str(Path(data.get('path', '../coco')) / 'annotations/instances_val2017.json')  # annotations json
        pred_json = str(save_dir / f"{w}_predictions.json")  # predictions json
        print(f'\nEvaluating pycocotools mAP... saving {pred_json}...')
        with open(pred_json, 'w') as f:
            json.dump(jdict, f)

        try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
            check_requirements(['pycocotools'])
            from pycocotools.coco import COCO
            from pycocotools.cocoeval import COCOeval

            anno = COCO(anno_json)  # init annotations api
            pred = anno.loadRes(pred_json)  # init predictions api
            eval = COCOeval(anno, pred, 'bbox')
            if is_coco:
                eval.params.imgIds = [int(Path(x).stem) for x in dataloader.dataset.img_files]  # image IDs to evaluate
            eval.evaluate()
            eval.accumulate()
            eval.summarize()
            map, map50 = eval.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)
        except Exception as e:
            print(f'pycocotools unable to run: {e}')

    # Return results
    model.float()  # for training
    if not training:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {colorstr('bold', save_dir)}{s}")
    maps = np.zeros(nc) + map
    for i, c in enumerate(ap_class):
        maps[c] = ap[i]
    return (mp, mr, map50, map, *(loss.cpu() / len(dataloader)).tolist()), maps, t,f2,conf_i

In [ ]:
results, maps, _,f2,conf_i = run()

In [ ]:
'''mp, mr, map50, map0.3:0.8'''
results 

In [ ]:
best_conf = conf_i*0.001
best_conf 

In [ ]:
f2

In [ ]:
'''1.Change yolov5/utils/metrics.py  ,ap_per_class()'''

In [ ]:
# '''
# def ap_per_class(tp, conf, pred_cls, target_cls, plot=False, save_dir='.', names=()):
#     """ Compute the average precision, given the recall and precision curves.
#     Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
#     # Arguments
#         tp:  True positives (nparray, nx1 or nx10).
#         conf:  Objectness value from 0-1 (nparray).
#         pred_cls:  Predicted object classes (nparray).
#         target_cls:  True object classes (nparray).
#         plot:  Plot precision-recall curve at mAP@0.5
#         save_dir:  Plot save directory
#     # Returns
#         The average precision as computed in py-faster-rcnn.
#     """

#     # Sort by objectness
#     i = np.argsort(-conf)
#     tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

#     # Find unique classes
#     unique_classes = np.unique(target_cls)
#     nc = unique_classes.shape[0]  # number of classes, number of detections

#     # Create Precision-Recall curve and compute AP for each class
#     px, py = np.linspace(0, 1, 1000), []  # for plotting
#     ap, p, r = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
#     m_f2 = []
#     for ci, c in enumerate(unique_classes):
#         i = pred_cls == c
#         n_l = (target_cls == c).sum()  # number of labels
#         n_p = i.sum()  # number of predictions

#         if n_p == 0 or n_l == 0:
#             continue
#         else:
#             # Accumulate FPs and TPs
#             fpc = (1 - tp[i]).cumsum(0)
#             tpc = tp[i].cumsum(0)

#             # Recall
#             recall = tpc / (n_l + 1e-16)  # recall curve
#             r[ci] = np.interp(-px, -conf[i], recall[:, 4], left=0)  # negative x, xp because xp decreases

#             # Precision
#             precision = tpc / (tpc + fpc)  # precision curve
#             p[ci] = np.interp(-px, -conf[i], precision[:, 4], left=1)  # p at pr_score

#             '''计算mean-f2'''
#             for j in range(tp.shape[1]):
#                 '''遍历iou 0.3~0.8，计算对应的f2'''
#                 # Recall
#                 recall = tpc / (n_l + 1e-16)  # recall curve
#                 current_p = np.interp(-px, -conf[i], precision[:, j], left=0)# negative x, xp because xp decreases

#                 # Precision
#                 precision = tpc / (tpc + fpc)  # precision curve
#                 current_r = np.interp(-px, -conf[i], recall[:, j], left=1)

#                 '''对应当前iou下的f2'''
#                 current_f2 = 5 * (current_p * current_r) / (4 * current_p + current_r + 1e-16)
#                 m_f2.append(current_f2)

#             '''此时m_f2尺寸为[11,1000]'''
#             m_f2 = np.array(m_f2)

#             '''在11的这个维度上取平均，得到F2-socre[0.3,0.8]'''
#             m_f2=np.mean(m_f2, axis=0)
#             '''变换形状，方便计算最大值'''
#             m_f2=m_f2.reshape(1, 1000)

#             # AP from recall-precision curve
#             for j in range(tp.shape[1]):
#                 ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])

#                 if plot and j == 0:
#                     py.append(np.interp(px, mrec, mpre))  # precision at mAP@0.5

#     # Compute F1 (harmonic mean of precision and recall)
#     f1 = 2 * p * r / (p + r + 1e-16)

#     if plot:
#         plot_pr_curve(px, py, ap, Path(save_dir) / 'PR_curve.png', names)
#         plot_mc_curve(px, f1, Path(save_dir) / 'F1_curve.png', names, ylabel='F1')
#         plot_mc_curve(px, m_f2, Path(save_dir) / 'meanF2_curve.png', names, ylabel='mF2')
#         plot_mc_curve(px, p, Path(save_dir) / 'P_curve.png', names, ylabel='Precision')
#         plot_mc_curve(px, r, Path(save_dir) / 'R_curve.png', names, ylabel='Recall')

#     # i = f1.mean(0).argmax()  # max F1 index
#     i = m_f2.mean(0).argmax()
#     return p[:, i], r[:, i], ap, m_f2[:, i], unique_classes.astype('int32'),i
# '''